<FONT size="5.9pt"> **CONSEQUENTIAL LCA OF BARK BIOREFINING WITH PRODUCTION OF HIGH STRENGTH MATERIALS**
</FONT> 

# 1. Set-up the directories <a class="anchor" id="section1"></a> 

It is advised to create a new folder in which the Jupyter Notebooks are downloaded from the GitHub repository and saved. A "results" folder will be created to store all outputs of the code (e.g., graphs, tables). This code uploads the ecoinvent database from a local directory. Therefore, a license for the ecoinvent is required and the database should be stored on your computer directly.

In [1]:
import os
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as npwetaherBIOREFINING
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# define the initial working directory
wdir = Path().resolve() # get the directory of the notebook
pdir = os.path.join(wdir,'projects')
print(f"Working Directory: {wdir}")

# set the BW project directory
pathlib.Path(pdir).mkdir(parents=True, exist_ok=True)
os.environ['BRIGHTWAY2_DIR'] = pdir
print('Project directory:', os.environ['BRIGHTWAY2_DIR'])

# set the directory to store the results
result_dir = os.path.join(wdir,'results')   
pathlib.Path(result_dir).mkdir(parents=True, exist_ok=True)
print('Results directory:', result_dir)

# set the directory in which the databases are stored
db_dir = os.path.join(wdir,'databases')
pathlib.Path(db_dir).mkdir(parents=True, exist_ok=True)
print('Databases directory:', db_dir)

Working Directory: /home/leabraud/Documents/LCAbark
Project directory: /home/leabraud/Documents/LCAbark/projects
Results directory: /home/leabraud/Documents/LCAbark/results
Databases directory: /home/leabraud/Documents/LCAbark/databases


# 2. Import background databases <a class="anchor" id="section2"></a>

The background databases used in this study correspond to ecoinvent 3.9 consequential and biosphere3.

In [3]:
import lca_algebraic as alg
import brightway2 as bw

# initiate the project / choose project name
alg.initProject('lca_bark_sept25') ### NAME OF THE PROJECT CAN BE CHANGED

# import the ecoinvent database from a local directory
alg.importDb("ecoinvent_3.9_conseq", str(db_dir + '/ecoinvent 3.9_consequential_ecoSpold02/datasets')) ## VERSION OF ECOINVENT TO BE CHANGED

# overview of the databases
alg.list_databases()

Using environment variable BRIGHTWAY2_DIR for data directory:
/home/leabraud/Documents/LCAbark/projects


/tmp/ipykernel_248644/2000980516.py:5: DeprecationWarning: DEPRECATED : Use bw2io.import_ecoinvent_release() instead
  alg.initProject('lca_bark_sept25') ### NAME OF THE PROJECT CAN BE CHANGED
/tmp/ipykernel_248644/2000980516.py:8: DeprecationWarning: DEPRECATED : Use the new bw2io.import_ecoinvent_release instead
  alg.importDb("ecoinvent_3.9_conseq", str(db_dir + '/ecoinvent 3.9_consequential_ecoSpold02/datasets')) ## VERSION OF ECOINVENT TO BE CHANGED
[WARNING] Database 'ecoinvent_3.9_conseq' has already been imported 


Biosphere database already present!!! No setup is needed


,backend,nb_activities,type
name,,,
biosphere3,sqlite,4709,biosphere
ecoinvent_3.9_conseq,sqlite,18856,background
Foreground DB,sqlite,82,foreground


# 3. Create foreground databases <a class="anchor" id="section3"></a>

In [4]:
USER_DB = 'Foreground DB' ## NAME OF THE FOREGROUND DATABASE CAN BE CHANGED
alg.resetDb(USER_DB)
alg.resetParams()

[WARNING] Db Foreground DB was here. Reseting it


## 3.1. Get ecoinvent datasets

In [5]:
# Choose which version of ecoinvent to perform the LCA calculations with
ei = 'ecoinvent_3.9_conseq'

In [6]:
# technopshere flows
grid_SE = alg.findActivity(name='market for electricity, high voltage', loc = 'SE', db_name=ei) 
grid_RER = alg.findActivity(name='market group for electricity, high voltage', loc = 'RER', db_name=ei) 
ethylacetate = alg.findActivity(name='ethyl acetate production', loc = 'RER', db_name=ei) 
CO2 = alg.findActivity(name='carbon dioxide production, liquid', loc = 'RER', db_name=ei)
sodiumbicarbonate = alg.findActivity(name='market for sodium bicarbonate', loc = 'GLO', db_name=ei)
sodiumhydroxide = alg.findActivity(name='market for sodium hydroxide, without water, in 50% solution state', loc = 'GLO', db_name=ei)
quicklime = alg.findActivity(name='quicklime production, milled, loose', loc = 'CH', db_name=ei)
wwt = alg.findActivity(name='treatment of wastewater, average, wastewater treatment', loc = 'Europe without Switzerland', db_name=ei)
mw_RER = alg.findActivity(name='market group for municipal solid waste', loc = 'RER', db_name=ei) 
heatwoodchips = alg.findActivity(name='heat production, wood chips from industry, at furnace 50kW', loc = 'CH', db_name=ei)
tapwater = alg.findActivity(name='market group for tap water', loc = 'RER', db_name=ei)
hydrogenperoxide = alg.findActivity(name='market for hydrogen peroxide, without water, in 50% solution state', loc='RER', db_name=ei)
sodiumhypochlorite = alg.findActivity(name='market for sodium hypochlorite, without water, in 15% solution state', loc='RER', db_name=ei)
ozone = alg.findActivity(name='market for ozone, liquid', loc='RER', db_name=ei)

# substituted products
talloil = alg.findActivity(name='tall oil refinery operation', loc = 'GLO', db_name=ei) 
potatostarch = alg.findActivity(name='potato starch production', loc = 'DE', db_name=ei) 
phenol = alg.findActivity(name='phenol production, from cumene', loc = 'RER', db_name=ei)
pulp = alg.findActivity(name='sulfate pulp production, from softwood, unbleached', loc = 'RER', db_name=ei)
limestone = alg.findActivity(name='market for limestone residue', loc = 'CH', db_name=ei)
kraftliner = alg.findActivity(name='market for kraft paper', loc = 'RER', db_name=ei)
PET = alg.findActivity(name='polyethylene terephthalate production, granulate, bottle grade', loc = 'RER', db_name=ei)
LDPE = alg. findActivity(name='packaging film production, low density polyethylene', loc='RER', db_name=ei)

# biosphere flows
ethylacetateemission = alg.findActivity(name='Ethyl acetate', db_name='biosphere3', categories = ('air',))
water = alg.findActivity(name='Water, river', db_name='biosphere3', categories = ('natural resource','in water'))
carbondioxide = alg.findActivity(name='Carbon dioxide, fossil', db_name='biosphere3', categories = ('air',))
carbondioxidenonfossil = alg.findActivity(name='Carbon dioxide, non-fossil', db_name='biosphere3', categories = ('air',))

## 3.2. Modify the ecoinvent dataset for heat production via cogeneration

The ecoinvent dataset for heat production from wood chips was used to model the environmental burdens of marginal heat supply in both biorefinery configurations. It was also used to calculate the impacts of bark combustion in the combustion scenario and residual biomass combustion in the biorefineries. The dataset was adapted for each specific application (see below).

In [7]:
# dataset selected in ecoinvent
heat = alg.findActivity(name='heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', loc = 'SE', db_name=ei)

### 3.2.1. Combustion of bark 

The new dataset "heat_CHP_bark" includes the impacts associated with the cogeneration process itself, excluding those from wood chip production (assumed similar to bark) and direct non-fossil emissions from wood combustion. The system boundaries exclude bark production (as this process is similar in all three assessed scenarios), while biogenic carbon emissions are accounted for separately. Since the properties of wood chips and bark are similar, it was assumed that combustion processes would result in the same impacts regardless of whether wood chips or bark are used as input.

In [8]:
heat_CHP_bark = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)
heat_CHP_bark.deleteExchanges('Carbon dioxide, non-fossil')
heat_CHP_bark.deleteExchanges('Carbon monoxide, non-fossil')
if ei == 'ecoinvent_3.9_conseq':
    heat_CHP_bark.deleteExchanges('wood chips, wet, measured as dry mass') # for ecoinvent 3.9
else:    
    heat_CHP_bark.deleteExchanges('market for wood chips, wet, measured as dry mass')
#alg.printAct(heat_CHP_bark)

### 3.2.2. Burdens from marginal heat production

The unmodified ecoinvent dataset was used as a basis. Here, the objective is to model the heat production from wood chips that would be required to compensate for the heat produced when bark is combusted in the business-as-usual scenario (i.e. combustion scenario). The total amount of heat that would need to be produced for compensation corresponds to the total amount of heat resulting from the combustion of 1 kg DW-eq of bark minus the heat produced via combustion of the residual biomass obtained in the biorefinery. The original process extracted from ecoinvent is not changed since the impacts from wood chips production and combustion are accounted for. The biogenic CO2 and CO from the cogeneration process are not removed. The original activity is simply copied and given a new name for distinction.

In [9]:
heat_CHP_woodchips = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)

[WARNING] Activity 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' was already in 'Foreground DB'. Overwriting it


### 3.2.3. Heat production from residual biomass

Again, the same ecoinvent dataset is used as a basis. The amount of residual biomass produced at each processing step is calculated from the mass and carbon balance. The amount of heat produced is calculated using the relation: 1 kg dry mass => 8.5 MJ according to the ecoinvent dataset for heat from cogeneration using wood chips. The same dataset as the one used for heat production from bark is used. No impacts from the biomass and no direct emissions are accounted for. A new name is given for distinction. 

In [10]:
heat_CHP_RB = alg.copyActivity(USER_DB, heat, code = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014', withExchanges=True)
heat_CHP_RB.deleteExchanges('Carbon dioxide, non-fossil')
heat_CHP_RB.deleteExchanges('Carbon monoxide, non-fossil')
if ei == 'ecoinvent_3.9_conseq':
    heat_CHP_RB.deleteExchanges('wood chips, wet, measured as dry mass') # ecoinvent 3.9
else:
    heat_CHP_RB.deleteExchanges('market for wood chips, wet, measured as dry mass')

[WARNING] Activity 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' was already in 'Foreground DB'. Overwriting it


In [11]:
# check the climae impacts of heat production using IPCC 2021
ipcc2021 = alg.findMethods('global warming potential (GWP100)', 'IPCC 2021')[0]
climate_impacts_heat_CHP_RB = alg.compute_impacts([heat_CHP_RB],[ipcc2021])
climate_impacts_heat_CHP_RB

[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia


,climate change - global warming potential (GWP100)[kg CO2-Eq]
"heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014[SE]",-0.0142485


# 4. Create foreground parameters <a class="anchor" id="section4"></a>

## 4.1. Evaluate the influence of the geographical location

In the sensititvity analysis, the influence of the geographical location on the LCA results is analysed. There are no European market for heat production from cogeneration in the consequential ecoinvent v.3.9 database. However, the dataset exists for many locations. The only differences between the datasets are the electricity mix and the municipal solid waste treatment. Here, both the electricity mix and the waste treatment processes are changed based on the geographical location. List of countries analysed in the sensitivity analysis: SE, NO, FI, DK, FR, DE, BE, LU, NL, AT.

In [12]:
# SE and RER have already been created above
grid_NO = alg.findActivity(name='market for electricity, high voltage', loc = 'NO', db_name=ei) 
grid_FI = alg.findActivity(name='market for electricity, high voltage', loc = 'FI', db_name=ei) 
grid_DK = alg.findActivity(name='market for electricity, high voltage', loc = 'DK', db_name=ei) 
grid_FR = alg.findActivity(name='market for electricity, high voltage', loc = 'FR', db_name=ei) 
grid_DE = alg.findActivity(name='market for electricity, high voltage', loc = 'DE', db_name=ei) 
grid_BE = alg.findActivity(name='market for electricity, high voltage', loc = 'BE', db_name=ei) 
grid_LU = alg.findActivity(name='market for electricity, high voltage', loc = 'LU', db_name=ei) 
grid_NL = alg.findActivity(name='market for electricity, high voltage', loc = 'NL', db_name=ei) 
grid_AT = alg.findActivity(name='market for electricity, high voltage', loc = 'AT', db_name=ei) 

# RER has already been created above
mw_SE = alg.findActivity(name='market for municipal solid waste', loc = 'SE', db_name=ei) 
mw_NO = alg.findActivity(name='market for municipal solid waste', loc = 'NO', db_name=ei) 
mw_FI = alg.findActivity(name='market for municipal solid waste', loc = 'FI', db_name=ei) 
mw_DK = alg.findActivity(name='market for municipal solid waste', loc = 'DK', db_name=ei) 
mw_FR = alg.findActivity(name='market for municipal solid waste', loc = 'FR', db_name=ei) 
mw_DE = alg.findActivity(name='market for municipal solid waste', loc = 'DE', db_name=ei) 
mw_BE = alg.findActivity(name='market for municipal solid waste', loc = 'BE', db_name=ei) 
mw_LU = alg.findActivity(name='market for municipal solid waste', loc = 'LU', db_name=ei) 
mw_NL = alg.findActivity(name='market for municipal solid waste', loc = 'NL', db_name=ei) 
mw_AT = alg.findActivity(name='market for municipal solid waste', loc = 'AT', db_name=ei) 

In [13]:
# create switch parameter for electricity mix - default value is SE
grid_loc = alg.newEnumParam(
    name = 'grid_loc',
    values = ['grid_SE','grid_NO','grid_FI','grid_DK','grid_FR','grid_DE','grid_BE','grid_LU','grid_NL','grid_AT','grid_RER'],
    default = 'grid_SE',
    #label = 'source of electricity - biorefinery',
    description = "Switch on the source of electricty used in the biorefinery",
    group = 'energy')

# create the virtual activity corresponding to the switch
grid_switch_act = alg.newSwitchAct(dbname = USER_DB,
                                   name = 'grid_switch_act',
                                   paramDef = grid_loc,
                                   acts_dict = {'grid_SE' : grid_SE,
                                                'grid_NO': grid_NO,
                                                'grid_FI': grid_FI,
                                                'grid_DK': grid_DK,
                                                'grid_FR': grid_FR,
                                                'grid_DE': grid_DE,
                                                'grid_BE': grid_BE,
                                                'grid_LU': grid_LU,
                                                'grid_NL': grid_NL,
                                                'grid_AT': grid_AT,
                                                'grid_RER': grid_RER})

In [14]:
# create switch parameter for municipal solid waste treatment - default value is RER
mw_loc = alg.newEnumParam(
    name = 'mw_loc',
    values = ['mw_SE','mw_NO','mw_FI','mw_DK','mw_FR','mw_DE','mw_BE','mw_LU','mw_NL','mw_AT','mw_RER'],
    default = 'mw_RER',
    #label = 'source of electricity - biorefinery',
    description = "Switch on the source of electricty used in the biorefinery",
    group = 'waste')

# create the virtual activity corresponding to the switch
mw_switch_act = alg.newSwitchAct(dbname = USER_DB,
                                   name = 'mw_switch_act',
                                   paramDef = mw_loc,
                                   acts_dict = {'mw_SE': mw_SE,
                                                'mw_NO': mw_NO,
                                                'mw_FI': mw_FI,
                                                'mw_DK': mw_DK,
                                                'mw_FR': mw_FR,
                                                'mw_DE': mw_DE,
                                                'mw_BE': mw_BE,
                                                'mw_LU': mw_LU,
                                                
                                                'mw_NL': mw_NL,
                                                'mw_AT': mw_AT,
                                                'mw_RER': mw_RER})

## 4.2. Evaluate the influence of the percentage of losses 

The float parameter defined corresponds to the percentage of biomass losses in the biorefinery processes. An increase in the percentage of losses means that less of the (co)products are produced but more heat is generated from the combustion of residual biomass.

In [15]:
percentage_losses = alg.newFloatParam(name = "percentage_losses",
                           default = 0, min = 0, max = 1, 
                           unit = '%',
                           group = 'biomass')

## 4.3. Evaluate the influence of the integration of the biorefinery with co-located source of energy

The boolean parameter defined changes the amount of electricity used in the biorefinery processes. In the integrated biorefinery (standalone = 0), no external electricity is required and all the energy is supplied from the co-located source which comes into the system free of burdens. In the standalone biorefinery (standalone = 1), the electricity required is supplied by the grid. 

In [16]:
standalone = alg.newBoolParam('standalone', 
                               default = 1, # 1 means the biorefinery system is stand-alone, 0 it is integrated
                               group = 'booleans')

# 5. Add the parameters to the foreground activities <a class="anchor" id="section5"></a>

## 5.1. Create parameterised heat production processes

The amount of electricity and municipal solid wastes are not changed, only the geographical location of the datasets. 

In [17]:
heat_CHP_bark.updateExchanges({'electricity*':grid_switch_act})
heat_CHP_bark.updateExchanges({'municipal*':mw_switch_act})

heat_CHP_woodchips.updateExchanges({'electricity*':grid_switch_act})
heat_CHP_woodchips.updateExchanges({'municipal*':mw_switch_act})

heat_CHP_RB.updateExchanges({'electricity*':grid_switch_act})
heat_CHP_RB.updateExchanges({'municipal*':mw_switch_act})

## 5.2. Create parameterised biorefinery processes

All the activities are scaled to the functional unit i.e. the processing of 1 kg of bark dry mass. 

In [18]:
# lipophilic extraction from bark
lipophilic_extraction = alg.newActivity(db_name = USER_DB, 
         name = 'lipophilic extraction',
         unit = 'unit',
         exchanges={
             # technosphere inputs
             ethylacetate: 0.002,
             grid_SE: 2.44*standalone,
             # emissions
             ethylacetateemission: 0.002
         })

lipophilic_extraction.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(lipophilic_extraction).to_excel(str(result_dir+'/lipophilic_extraction_'+ei+'.xlsx'))
alg.printAct(lipophilic_extraction)

lipophilic extraction (1.000000 unit)  \
                                                                     input   
Ethyl acetate                                                Ethyl acetate   
ethyl acetate production                     ethyl acetate production[RER]   
market for electricity, high voltage                   grid_switch_act{FG}   

                                                                      
                                               amount           unit  
Ethyl acetate                                   0.002       kilogram  
ethyl acetate production                        0.002       kilogram  
market for electricity, high voltage  2.44*standalone  kilowatt hour

In [19]:
# sugar extraction from lipophilic depleted bark
sugar_extraction = alg.newActivity(db_name = USER_DB, 
 name = 'sugar extraction',
 unit = 'unit',
 exchanges={
     grid_SE: 5*standalone,
     wwt: -0.00007,
     # natural resources
     water: 0.00007
 })

sugar_extraction.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(sugar_extraction).to_excel(str(result_dir+'/sugar_extraction_'+ei+'.xlsx'))
alg.printAct(sugar_extraction)

sugar extraction (1.000000 unit)  \
                                                                                                input   
Water, river                                                                             Water, river   
market for electricity, high voltage                                              grid_switch_act{FG}   
treatment of wastewater, average, wastewater tr...  treatment of wastewater, average, wastewater t...   

                                                                  \
                                                          amount   
Water, river                                               7e-05   
market for electricity, high voltage                5*standalone   
treatment of wastewater, average, wastewater tr...        -7e-05   

                                                                   
                                                             unit  
Water, river                                          cubic meter  
market for electricity, high voltage                kilowatt hour  
treatment of wastewater, average, wastewater tr...    cubic meter

In [20]:
sugar_extraction_withlosses = alg.newActivity(db_name = USER_DB,
                                     name = 'sugar_extraction_withlosses',
                                     unit = 'unit',
                                     exchanges={sugar_extraction:1 # - percentage_losses
                                    })

In [21]:
# pulping of sugar depleted bark
pulping = alg.newActivity(db_name = USER_DB, 
 name = 'pulping',
 unit = 'unit',
 exchanges={
     sodiumhydroxide: 0.00523,
     quicklime: 0.00629, # calcium oxide
     wwt: -0.00035,
     grid_SE: 1.03*standalone,
     heatwoodchips:0.35,
     CO2: 0.124*standalone,
     limestone:-0.00551,
     # natural resources
     water: 0.00035
 })

pulping.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(pulping).to_excel(str(result_dir+'/pulping_'+ei+'.xlsx'))
alg.printAct(pulping)

pulping (1.000000 unit)  \
                                                                                                input   
Water, river                                                                             Water, river   
carbon dioxide production, liquid                              carbon dioxide production, liquid[RER]   
heat production, wood chips from industry, at f...  heat production, wood chips from industry, at ...   
market for electricity, high voltage                                              grid_switch_act{FG}   
market for limestone residue                                         market for limestone residue[CH]   
market for sodium hydroxide, without water, in ...  market for sodium hydroxide, without water, in...   
quicklime production, milled, loose                           quicklime production, milled, loose[CH]   
treatment of wastewater, average, wastewater tr...  treatment of wastewater, average, wastewater t...   

                                                                      \
                                                              amount   
Water, river                                                 0.00035   
carbon dioxide production, liquid                   0.124*standalone   
heat production, wood chips from industry, at f...              0.35   
market for electricity, high voltage                 1.03*standalone   
market for limestone residue                                -0.00551   
market for sodium hydroxide, without water, in ...           0.00523   
quicklime production, milled, loose                          0.00629   
treatment of wastewater, average, wastewater tr...          -0.00035   

                                                                   
                                                             unit  
Water, river                                          cubic meter  
carbon dioxide production, liquid                        kilogram  
heat production, wood chips from industry, at f...      megajoule  
market for electricity, high voltage                kilowatt hour  
market for limestone residue                             kilogram  
market for sodium hydroxide, without water, in ...       kilogram  
quicklime production, milled, loose                      kilogram  
treatment of wastewater, average, wastewater tr...    cubic meter

In [22]:
pulping_withlosses = alg.newActivity(db_name = USER_DB,
                                     name = 'pulping_withlosses',
                                     unit = 'unit',
                                     exchanges={pulping:1 #- percentage_losses
                                               })

In [23]:
# coated Lig-MFC (70 MPa)
coatedLigMFC = alg.newActivity(db_name = USER_DB, 
 name = 'coatedLigMFC',
 unit = 'unit',
 exchanges={
     tapwater: 0.55,
     grid_SE: 2.175*standalone,
 })

coatedLigMFC.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(coatedLigMFC).to_excel(str(result_dir+'/pulping_'+ei+'.xlsx'))
alg.printAct(coatedLigMFC)

coatedLigMFC (1.000000 unit)  \
                                                                input   
market for electricity, high voltage              grid_switch_act{FG}   
market group for tap water            market group for tap water[RER]   

                                                                       
                                                amount           unit  
market for electricity, high voltage  2.175*standalone  kilowatt hour  
market group for tap water                        0.55       kilogram

In [24]:
# coatedbleachMFC (170 MPa)
bleachcoatMFC = alg.newActivity(db_name = USER_DB, 
 name = 'bleachcoatMFC',
 unit = 'unit',
 exchanges={
     hydrogenperoxide:0.024,
     ozone:0.066*0.059,
     sodiumhypochlorite:0.215*0.059,
     tapwater:0.28,
     grid_SE: 2.32*standalone,
 })

bleachcoatMFC.updateExchanges({'electricity*':grid_switch_act})
#alg.printAct(coatedbleachMFC).to_excel(str(result_dir+'/pulping_'+ei+'.xlsx'))
alg.printAct(bleachcoatMFC)

bleachcoatMFC (1.000000 unit)  \
                                                                                                input   
market for electricity, high voltage                                              grid_switch_act{FG}   
market for hydrogen peroxide, without water, in...  market for hydrogen peroxide, without water, i...   
market for ozone, liquid                                                market for ozone, liquid[RER]   
market for sodium hypochlorite, without water, ...  market for sodium hypochlorite, without water,...   
market group for tap water                                            market group for tap water[RER]   

                                                                     \
                                                             amount   
market for electricity, high voltage                2.32*standalone   
market for hydrogen peroxide, without water, in...            0.024   
market for ozone, liquid                                   0.003894   
market for sodium hypochlorite, without water, ...         0.012685   
market group for tap water                                     0.28   

                                                                   
                                                             unit  
market for electricity, high voltage                kilowatt hour  
market for hydrogen peroxide, without water, in...       kilogram  
market for ozone, liquid                                 kilogram  
market for sodium hypochlorite, without water, ...       kilogram  
market group for tap water                               kilogram

In [25]:
# production of heat from residual biomass via cogeneration
heat_residues = alg.newActivity(db_name = USER_DB, 
 name = 'heat (RB)',
 unit = 'unit',
 exchanges={
     heat_CHP_RB: 1.53*(1+percentage_losses)
 })
alg.printAct(heat_residues)

heat (RB) (1.000000 unit)  \
                                                                                                input   
heat and power co-generation, wood chips, 6667 ...  heat and power co-generation, wood chips, 6667...   

                                                                                   \
                                                                           amount   
heat and power co-generation, wood chips, 6667 ...  1.53*percentage_losses + 1.53   

                                                               
                                                         unit  
heat and power co-generation, wood chips, 6667 ...  megajoule

In [26]:
# conversion of lipophilic extractives into tall oil (no impacts assumed)
processA = alg.newActivity(db_name = USER_DB, 
 name = 'process A',
 unit = 'unit',
 exchanges={
 })

In [27]:
# conversion of sugar extractives into starch (no impacts assumed)
processB = alg.newActivity(db_name = USER_DB, 
 name = 'process B',
 unit = 'unit',
 exchanges={
 })

In [28]:
# conversion of lignin into phenol (no impacts assumed)
processC = alg.newActivity(db_name = USER_DB, 
 name = 'process C',
 unit = 'unit',
 exchanges={
 })

## 5.3. Create parameterised substituted processes

In [29]:
# substitution of tall oil from pine wood (1:1)
talloil_sub = alg.newActivity(db_name = USER_DB,
                             name = 'tall oil (sub.)',
                             unit = 'unit',
                             exchanges = {talloil: - ((0.02/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(talloil_sub)

tall oil (sub.) (1.000000 unit)  \
                                                      input   
tall oil refinery operation     tall oil refinery operation   

                                                                           \
                                                                   amount   
tall oil refinery operation  0.00439024390243902*percentage_losses - 0.02   

                                       
                                 unit  
tall oil refinery operation  kilogram

In [30]:
# substitution of starch from potato (1:1)
starch_sub = alg.newActivity(db_name = USER_DB,
                          name = 'starch (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.22/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(starch_sub)

starch (sub.) (1.000000 unit)  \
                                                 input   
potato starch production  potato starch production[DE]   

                                                                       \
                                                               amount   
potato starch production  0.0482926829268293*percentage_losses - 0.22   

                                    
                              unit  
potato starch production  kilogram

In [31]:
# substitution of phenol from cumene (1:1)
phenol_sub = alg.newActivity(db_name = USER_DB,
                            name = 'phenol (sub.)',
                            unit = 'unit',
                            exchanges = {phenol: -((0.17/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(phenol_sub)

phenol (sub.) (1.000000 unit)  \
                                                              input   
phenol production, from cumene  phenol production, from cumene[RER]   

                                                                             \
                                                                     amount   
phenol production, from cumene  0.0373170731707317*percentage_losses - 0.17   

                                          
                                    unit  
phenol production, from cumene  kilogram

In [32]:
# substitution of sulfate pulp (1:1)
pulp_sub = alg.newActivity(db_name = USER_DB,
                            name = 'pulp (sub.)',
                            unit = 'unit',
                            exchanges = {pulp: -((0.41/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(pulp_sub)

pulp (sub.) (1.000000 unit)  \
                                                                                                input   
sulfate pulp production, from softwood, unbleached  sulfate pulp production, from softwood, unblea...   

                                                                                   \
                                                                           amount   
sulfate pulp production, from softwood, unbleached  0.09*percentage_losses - 0.41   

                                                              
                                                        unit  
sulfate pulp production, from softwood, unbleached  kilogram

In [33]:
# environmental impacts from the production of marginal heat in the biorefinery scenarios (e.g., heat that would have been produced by directly burning the bark)
# amount corresponds to the total amount of heat produced when burning bark (8.5 MJ) minus the heat produced from the residues
heat_burdens = alg.newActivity(db_name = USER_DB, 
 name = 'heat (burd.)',
 unit = 'unit',
 exchanges={
     heat_CHP_woodchips: 8.5 -(1.53*(1+percentage_losses))
 })
alg.printAct(heat_burdens)

heat (burd.) (1.000000 unit)  \
                                                                                                input   
heat and power co-generation, wood chips, 6667 ...  heat and power co-generation, wood chips, 6667...   

                                                                                   \
                                                                           amount   
heat and power co-generation, wood chips, 6667 ...  6.97 - 1.53*percentage_losses   

                                                               
                                                         unit  
heat and power co-generation, wood chips, 6667 ...  megajoule

In [34]:
# substitution of kraft liner by coatLigMF
kraftliner_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'Kraft kiner coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {kraftliner: -0.99})
alg.printAct(kraftliner_sub_coatLigMF)

Kraft kiner coatLigMF (sub.) (1.000000 unit)         \
                                                              input amount   
market for kraft paper                  market for kraft paper[RER]  -0.99   

                                  
                            unit  
market for kraft paper  kilogram

In [35]:
# substitution of PET by coatLigMF
PET_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'PET coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {PET: -0.46})
alg.printAct(PET_sub_coatLigMF)

PET coatLigMF (sub.) (1.000000 unit)  \
                                                                                                input   
polyethylene terephthalate production, granulat...  polyethylene terephthalate production, granula...   

                                                                     
                                                   amount      unit  
polyethylene terephthalate production, granulat...  -0.46  kilogram

In [36]:
# substitution of LDPE by coatLigMF
LDPE_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'LDPE coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {LDPE: -2.44})
alg.printAct(LDPE_sub_coatLigMF)

LDPE coatLigMF (sub.) (1.000000 unit)  \
                                                                                                input   
packaging film production, low density polyethy...  packaging film production, low density polyeth...   

                                                                     
                                                   amount      unit  
packaging film production, low density polyethy...  -2.44  kilogram

In [37]:
# substitution of sulfate pulp (1:1) in coatLigMF scenario
pulp_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                            name = 'pulp coatLigMF (sub.)',
                            unit = 'unit',
                            exchanges = {pulp: -((0.02/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(pulp_sub_coatLigMF)

pulp coatLigMF (sub.) (1.000000 unit)  \
                                                                                                input   
sulfate pulp production, from softwood, unbleached  sulfate pulp production, from softwood, unblea...   

                                                                                                  \
                                                                                          amount   
sulfate pulp production, from softwood, unbleached  0.00439024390243902*percentage_losses - 0.02   

                                                              
                                                        unit  
sulfate pulp production, from softwood, unbleached  kilogram

In [38]:
# substitution of starch from potato (1:1) in coatLigMF scenario
starch_sub_coatLigMF = alg.newActivity(db_name = USER_DB,
                          name = 'starch coatLigMF (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.03/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(starch_sub_coatLigMF)

starch coatLigMF (sub.) (1.000000 unit)  \
                                                           input   
potato starch production            potato starch production[DE]   

                                                                        \
                                                                amount   
potato starch production  0.00658536585365854*percentage_losses - 0.03   

                                    
                              unit  
potato starch production  kilogram

In [39]:
# substitution of kraft liner by bleachcoatMFC
kraftliner_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'Kraft kiner bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {kraftliner: -1.12})
alg.printAct(kraftliner_sub_bleachcoatMFC)

Kraft kiner bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                  input   
market for kraft paper                      market for kraft paper[RER]   

                                         
                       amount      unit  
market for kraft paper  -1.12  kilogram

In [40]:
# substitution of PET bleachcoatMFC
PET_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'PET bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {PET: -0.53})
alg.printAct(PET_sub_bleachcoatMFC)

PET bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                                                input   
polyethylene terephthalate production, granulat...  polyethylene terephthalate production, granula...   

                                                                     
                                                   amount      unit  
polyethylene terephthalate production, granulat...  -0.53  kilogram

In [41]:
# substitution of LDPE by bleachcoatMFC
LDPE_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'LDPE bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {LDPE: -2.67})
alg.printAct(LDPE_sub_bleachcoatMFC)

LDPE bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                                                input   
packaging film production, low density polyethy...  packaging film production, low density polyeth...   

                                                                     
                                                   amount      unit  
packaging film production, low density polyethy...  -2.67  kilogram

In [42]:
# substitution of heat by residues from bleachcoatMFC process
heat_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                            name = 'heat bleachcoatMFC (sub.)',
                            unit = 'unit',
                            exchanges = {heat_CHP_woodchips: -1.74})
alg.printAct(heat_sub_bleachcoatMFC)

heat bleachcoatMFC (sub.) (1.000000 unit)  \
                                                                                                input   
heat and power co-generation, wood chips, 6667 ...  heat and power co-generation, wood chips, 6667...   

                                                                      
                                                   amount       unit  
heat and power co-generation, wood chips, 6667 ...  -1.74  megajoule

In [43]:
# substitution of starch from potato (1:1) in coatLigMF scenario
starch_sub_bleachcoatMFC = alg.newActivity(db_name = USER_DB,
                          name = 'starch bleachcoatMFC (sub.)',
                          unit = 'unit',
                          exchanges = {potatostarch: -((0.144/0.82)*(1-0.18*(1+percentage_losses)))})
alg.printAct(starch_sub_bleachcoatMFC)

starch bleachcoatMFC (sub.) (1.000000 unit)  \
                                                               input   
potato starch production                potato starch production[DE]   

                                                                       \
                                                               amount   
potato starch production  0.031609756097561*percentage_losses - 0.144   

                                    
                              unit  
potato starch production  kilogram

## 5.4. Scaled process for combustion scenario

In [44]:
heat_CHP_bark_scaled = alg.newActivity(db_name = USER_DB,
                                     name = 'heat CHP bark',
                                     unit = 'unit',
                                     exchanges={heat_CHP_bark:8.5})

# 6. Calculate the static climate impacts using IPCC 2021 GWP100

In [45]:
df_bioref_standalone_coatedLigMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            kraftliner_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia
[INFO] Db changed recently, clearing cache expr
[INFO] Db changed recently, clearing cache lcia
[INFO] Db changed recently, clearing cache expr


In [46]:
df_bioref_standalone_coatedLigMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            PET_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [47]:
df_bioref_standalone_coatedLigMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            LDPE_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [48]:
df_bioref_standalone_bleachcoatMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            kraftliner_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [49]:
df_bioref_standalone_bleachcoatMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            PET_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [50]:
df_bioref_standalone_bleachcoatMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            LDPE_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 1,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [51]:
df_bioref_integrated_coatedLigMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            kraftliner_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [52]:
df_bioref_integrated_coatedLigMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            PET_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [53]:
df_bioref_integrated_coatedLigMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            coatedLigMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            pulp_sub_coatLigMF,
                                            starch_sub_coatLigMF,
                                            LDPE_sub_coatLigMF],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [54]:
df_bioref_integrated_bleachcoatMFC_KL = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            kraftliner_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [55]:
df_bioref_integrated_bleachcoatMFC_PET = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            PET_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [56]:
df_bioref_integrated_bleachcoatMFC_LDPE = alg.compute_impacts([
                                            # processes
                                            lipophilic_extraction, 
                                            sugar_extraction_withlosses, 
                                            pulping_withlosses, 
                                            bleachcoatMFC,
                                            heat_burdens, 
                                            heat_residues,                                 
                                            # substitution
                                            talloil_sub,
                                            phenol_sub, 
                                            heat_sub_bleachcoatMFC,
                                            starch_sub_bleachcoatMFC,
                                            LDPE_sub_bleachcoatMFC],
                                            # lcia method
                                            [ipcc2021],
                                            # parameters
                                            standalone = 0,
                                            percentage_losses = 0,
                                            grid_loc = 'grid_SE',
                                            mw_loc = 'mw_RER'
                                          )

In [57]:
# static climate impacts (indirect emissions only) for the combustion scenario
df_combustion = alg.compute_impacts([heat_CHP_bark_scaled],
                            [ipcc2021],
                            # parameters
                            percentage_losses = 0,
                            grid_loc = 'grid_SE',
                            mw_loc = 'mw_RER'
                            )
#df_combustion.to_excel(str(result_dir+"/climate_impacts_IPCC2021_static_combustion_" + str(ei)+ ".xlsx"))

In [58]:
# merge the three dataframes
merged_df = pd.concat([df_combustion, 
                       df_bioref_standalone_coatedLigMFC_KL, 
                       df_bioref_standalone_coatedLigMFC_PET, 
                       df_bioref_standalone_coatedLigMFC_LDPE, 
                       df_bioref_standalone_bleachcoatMFC_KL, 
                       df_bioref_standalone_bleachcoatMFC_PET, 
                       df_bioref_standalone_bleachcoatMFC_LDPE, 
                       df_bioref_integrated_coatedLigMFC_KL, 
                       df_bioref_integrated_coatedLigMFC_PET, 
                       df_bioref_integrated_coatedLigMFC_LDPE, 
                       df_bioref_integrated_bleachcoatMFC_KL, 
                       df_bioref_integrated_bleachcoatMFC_PET, 
                       df_bioref_integrated_bleachcoatMFC_LDPE, 
                       ], 
                      axis = 1)
# change the name of the columns
merged_df.columns = ['Combustion',
                     'Standalone coatedLigMFC KL',
                     'Standalone coatedLigMFC PET',
                     'Standalone coatedLigMFC LDPE',
                     'Standalone bleachcoatMFC KL',
                     'Standalone bleachcoatMFC PET',
                     'Standalone bleachcoatMFC LDPE',
                     'Integrated coatedLigMFC KL',
                     'Integrated coatedLigMFC PET',
                     'Integrated coatedLigMFC LDPE',
                     'Integrated bleachcoatMFC KL',
                     'Integrated bleachcoatMFC PET',
                     'Integrated bleachcoatMFC LDPE',
                    ]
merged_df

,Combustion,Standalone coatedLigMFC KL,Standalone coatedLigMFC PET,Standalone coatedLigMFC LDPE,Standalone bleachcoatMFC KL,Standalone bleachcoatMFC PET,Standalone bleachcoatMFC LDPE,Integrated coatedLigMFC KL,Integrated coatedLigMFC PET,Integrated coatedLigMFC LDPE,Integrated bleachcoatMFC KL,Integrated bleachcoatMFC PET,Integrated bleachcoatMFC LDPE
heat CHP bark,-0.121089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lipophilic extraction,NaN,0.449329,0.449329,0.449329,0.449329,0.449329,0.449329,0.00927422,0.00927422,0.00927422,0.00927422,0.00927422,0.00927422
sugar_extraction_withlosses,NaN,0.901768,0.901768,0.901768,0.901768,0.901768,0.901768,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05,1.6207e-05
pulping_withlosses,NaN,0.353132,0.353132,0.353132,0.353132,0.353132,0.353132,0.034189,0.034189,0.034189,0.034189,0.034189,0.034189
coatedLigMFC,NaN,0.392526,0.392526,0.392526,NaN,NaN,NaN,0.000263937,0.000263937,0.000263937,NaN,NaN,NaN
heat (burd.),NaN,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929,-0.0992929
heat (RB),NaN,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796,-0.021796
tall oil (sub.),NaN,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491,-0.0366491
phenol (sub.),NaN,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767,-0.401767
pulp coatLigMF (sub.),NaN,0.00136126,0.00136126,0.00136126,NaN,NaN,NaN,0.00136126,0.00136126,0.00136126,NaN,NaN,NaN


In [59]:
merged_df.loc["Total"] = merged_df.sum()

In [60]:
merged_df.to_excel('results_comparison_coatLigMFC_bleachcoatMF.xlsx')